## Predicting Absenteeism at Work

Given *data about absenteeism of employees*, let's try to predict **how many hours** a given employee will be absent. 

We will use various regression models to make our predictions. 

Data source: https://www.kaggle.com/datasets/kewagbln/absenteeism-at-work-uci-ml-repositiory

### Importing Libraries

In [1]:
# !pip install lightgbm

In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
pd.set_option("display.max_columns", None)

In [6]:
data = pd.read_csv("Absenteeism_at_work.csv", delimiter=";")
data

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,97,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,97,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,97,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,11,14,7,3,1,289,36,13,33,264.604,93,0,1,2,1,0,1,90,172,30,8
736,1,11,7,3,1,235,11,14,37,264.604,93,0,3,1,0,0,1,88,172,29,4
737,4,0,0,3,1,118,14,13,40,271.219,95,0,1,1,1,0,8,98,170,34,0
738,8,0,0,4,2,231,35,14,39,271.219,95,0,1,2,1,0,2,100,170,35,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID                               740 non-null    int64  
 1   Reason for absence               740 non-null    int64  
 2   Month of absence                 740 non-null    int64  
 3   Day of the week                  740 non-null    int64  
 4   Seasons                          740 non-null    int64  
 5   Transportation expense           740 non-null    int64  
 6   Distance from Residence to Work  740 non-null    int64  
 7   Service time                     740 non-null    int64  
 8   Age                              740 non-null    int64  
 9   Work load Average/day            740 non-null    float64
 10  Hit target                       740 non-null    int64  
 11  Disciplinary failure             740 non-null    int64  
 12  Education             

### Preprocessing

In [8]:
df = data.copy()

In [9]:
# Drop ID column
df = df.drop('ID', axis=1)

In [10]:
def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [11]:
# One hot encode the reason column
df = onehot_encode(
    df,
    column='Reason for absence',
    prefix='Reason'
)

In [12]:
df

,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours,Reason_0,Reason_1,Reason_2,Reason_3,Reason_4,Reason_5,Reason_6,Reason_7,Reason_8,Reason_9,Reason_10,Reason_11,Reason_12,Reason_13,Reason_14,Reason_15,Reason_16,Reason_17,Reason_18,Reason_19,Reason_21,Reason_22,Reason_23,Reason_24,Reason_25,Reason_26,Reason_27,Reason_28
0,7,3,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1,7,3,1,118,13,18,50,239.554,97,1,1,1,1,0,0,98,178,31,0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,7,4,1,179,51,18,38,239.554,97,0,1,0,1,0,0,89,170,31,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
3,7,5,1,279,5,14,39,239.554,97,0,1,2,1,1,0,68,168,24,4,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,7,5,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,7,3,1,289,36,13,33,264.604,93,0,1,2,1,0,1,90,172,30,8,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
736,7,3,1,235,11,14,37,264.604,93,0,3,1,0,0,1,88,172,29,4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
737,0,3,1,118,14,13,40,271.219,95,0,1,1,1,0,8,98,170,34,0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
738,0,4,2,231,35,14,39,271.219,95,0,1,2,1,0,2,100,170,35,0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [13]:
# Split df into X and y
y = df['Absenteeism time in hours'].copy()
X = df.drop('Absenteeism time in hours', axis=1)

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)

In [14]:
X_train

,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Reason_0,Reason_1,Reason_2,Reason_3,Reason_4,Reason_5,Reason_6,Reason_7,Reason_8,Reason_9,Reason_10,Reason_11,Reason_12,Reason_13,Reason_14,Reason_15,Reason_16,Reason_17,Reason_18,Reason_19,Reason_21,Reason_22,Reason_23,Reason_24,Reason_25,Reason_26,Reason_27,Reason_28
664,4,4,3,118,10,10,37,246.288,91,0,1,0,0,0,0,83,172,28,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
547,11,3,4,225,26,9,28,268.519,93,0,1,1,0,0,2,69,169,24,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
22,8,2,1,361,52,3,28,205.917,92,0,1,1,1,0,4,80,172,27,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
283,9,4,4,235,20,13,43,294.217,81,0,1,1,1,0,0,106,167,38,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
145,2,3,2,388,15,9,50,302.585,99,0,1,0,0,0,0,76,178,24,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,12,3,4,248,25,14,47,261.306,97,0,1,2,0,0,1,86,165,32,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
322,11,2,4,378,49,11,36,284.031,97,0,1,2,0,1,4,65,174,21,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
382,2,3,2,246,25,16,41,251.818,96,0,1,0,1,0,0,67,170,23,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
365,1,2,2,235,20,13,43,330.061,100,0,1,1,1,0,0,106,167,38,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False


In [15]:
y_train

664     2
547     2
22     40
283     8
145     8
       ..
98      1
322     8
382     2
365     8
510     3
Name: Absenteeism time in hours, Length: 518, dtype: int64

In [18]:
# Scale X
scaler = StandardScaler()
scaler.fit(X_train)

X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

In [19]:
X_train

,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Reason_0,Reason_1,Reason_2,Reason_3,Reason_4,Reason_5,Reason_6,Reason_7,Reason_8,Reason_9,Reason_10,Reason_11,Reason_12,Reason_13,Reason_14,Reason_15,Reason_16,Reason_17,Reason_18,Reason_19,Reason_21,Reason_22,Reason_23,Reason_24,Reason_25,Reason_26,Reason_27,Reason_28
0,-0.703899,0.095346,0.395804,-1.521214,-1.311165,-0.560439,0.115190,-0.622623,-0.961304,-0.225189,-0.432536,-0.907178,-1.127799,-0.277350,-0.555464,0.318821,-0.043828,0.333984,-0.234499,-0.140303,-0.04398,-0.04398,-0.04398,-0.062257,-0.117041,-0.147296,-0.076323,-0.062257,5.601339,-0.184207,-0.098725,-0.293179,-0.147296,-0.062257,-0.076323,-0.04398,-0.195131,-0.239046,-0.076323,-0.234499,-0.522233,-0.062257,-0.200401,-0.220416,-0.300876,-0.427374
1,1.305817,-0.610216,1.287224,0.066665,-0.240214,-0.784614,-1.309258,-0.042983,-0.429016,-0.225189,-0.432536,0.001755,-1.127799,-0.277350,0.947044,-0.793831,-0.540974,-0.628484,-0.234499,-0.140303,-0.04398,-0.04398,-0.04398,-0.062257,-0.117041,-0.147296,-0.076323,-0.062257,-0.178529,-0.184207,-0.098725,-0.293179,-0.147296,-0.062257,-0.076323,-0.04398,-0.195131,-0.239046,-0.076323,-0.234499,1.914854,-0.062257,-0.200401,-0.220416,-0.300876,-0.427374
2,0.444510,-1.315778,-1.387035,2.084905,1.500080,-2.129668,-1.309258,-1.675237,-0.695160,-0.225189,-0.432536,0.001755,0.886683,-0.277350,2.449553,0.080396,-0.043828,0.093367,-0.234499,-0.140303,-0.04398,-0.04398,-0.04398,-0.062257,-0.117041,-0.147296,-0.076323,-0.062257,-0.178529,-0.184207,-0.098725,3.410886,-0.147296,-0.062257,-0.076323,-0.04398,-0.195131,-0.239046,-0.076323,-0.234499,-0.522233,-0.062257,-0.200401,-0.220416,-0.300876,-0.427374
3,0.731612,0.095346,1.287224,0.215065,-0.641821,0.112088,1.064822,0.627054,-3.622746,-0.225189,-0.432536,0.001755,0.886683,-0.277350,-0.555464,2.146751,-0.872404,2.740152,-0.234499,-0.140303,-0.04398,-0.04398,-0.04398,-0.062257,-0.117041,-0.147296,-0.076323,-0.062257,-0.178529,-0.184207,-0.098725,-0.293179,-0.147296,-0.062257,-0.076323,-0.04398,-0.195131,-0.239046,-0.076323,-0.234499,-0.522233,-0.062257,-0.200401,4.536886,-0.300876,-0.427374
4,-1.278104,-0.610216,-0.495616,2.485584,-0.976493,-0.784614,2.172726,0.845237,1.167849,-0.225189,-0.432536,-0.907178,-1.127799,-0.277350,-0.555464,-0.237505,0.950463,-0.628484,-0.234499,7.127412,-0.04398,-0.04398,-0.04398,-0.062257,-0.117041,-0.147296,-0.076323,-0.062257,-0.178529,-0.184207,-0.098725,-0.293179,-0.147296,-0.062257,-0.076323,-0.04398,-0.195131,-0.239046,-0.076323,-0.234499,-0.522233,-0.062257,-0.200401,-0.220416,-0.300876,-0.427374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,1.592919,-0.610216,1.287224,0.407985,-0.307149,0.336263,1.697910,-0.231051,0.635560,-0.225189,-0.432536,0.910688,-1.127799,-0.277350,0.195790,0.557247,-1.203834,1.296451,-0.234499,-0.140303,-0.04398,-0.04398,-0.04398,-0.062257,-0.117041,-0.147296,-0.076323,-0.062257,-0.178529,-0.184207,-0.098725,-0.293179,-0.147296,-0.062257,-0.076323,-0.04398,-0.195131,-0.239046,-0.076323,-0.234499,1.914854,-0.062257,-0.200401,-0.220416,-0.300876,-0.427374
514,1.305817,-1.315778,1.287224,2.337184,1.299277,-0.336263,-0.043082,0.361469,0.635560,-0.225189,-0.432536,0.910688,-1.127799,3.605551,2.449553,-1.111732,0.287602,-1.350334,-0.234499,7.127412,-0.04398,-0.04398,-0.04398,-0.062257,-0.117041,-0.147296,-0.076323,-0.062257,-0.178529,-0.184207,-0.098725,-0.293179,-0.147296,-0.062257,-0.076323,-0.04398,-0.195131,-0.239046,-0.076323,-0.234499,-0.522233,-0.062257,-0.200401,-0.220416,-0.300876,-0.427374
515,-1.278104,-0.610216,-0.495616,0.378305,-0.307149,0.784614,0.748278,-0.478437,0.369416,-0.225189,-0.432536,-0.907178,0.886683,-0.277350,-0.555464

In [21]:
X_train.describe()

,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Reason_0,Reason_1,Reason_2,Reason_3,Reason_4,Reason_5,Reason_6,Reason_7,Reason_8,Reason_9,Reason_10,Reason_11,Reason_12,Reason_13,Reason_14,Reason_15,Reason_16,Reason_17,Reason_18,Reason_19,Reason_21,Reason_22,Reason_23,Reason_24,Reason_25,Reason_26,Reason_27,Reason_28
count,5.180000e+02,5.180000e+02,5.180000e+02,518.000000,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,518.000000,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02,5.180000e+02
mean,2.057556e-17,-6.858521e-18,2.743408e-17,0.000000,-3.086334e-17,-8.573151e-18,6.858521e-18,-2.057556e-17,-3.429260e-17,-8.401688e-17,-5.486816e-17,-3.429260e-18,-2.743408e-17,-1.886093e-17,-5.829743e-17,-3.429260e-18,-1.371704e-17,-4.458038e-17,0.000000,6.858521e-18,-3.429260e-18,-3.429260e-18,-1.028778e-17,3.429260e-17,2.571945e-17,-6.858521e-18,1.200241e-17,3.429260e-17,6.858521e-17,1.371704e-17,8.573151e-18,3.257797e-17,-4.286575e-18,2.743408e-17,1.028778e-17,-3.429260e-18,-9.601929e-17,2.743408e-17,-5.143890e-18,3.429260e-17,-4.115112e-17,3.429260e-17,-1.063071e-16,3.429260e-17,-6.858521e-18,-2.057556e-17
std,1.000967e+00,1.000967e+00,1.000967e+00,1.000967,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00,1.000967e+00
min,-1.852309e+00,-1.315778e+00,-1.387035e+00,-1.521214,-1.645837e+00,-2.578019e+00,-1.467530e+00,-1.675237e+00,-3.622746e+00,-2.251887e-01,-4.325358e-01,-9.071781e-01,-1.127799e+00,-2.773501e-01,-5.554640e-01,-1.827009e+00,-1.535264e+00,-1.831568e+00,-0.234499,-1.403034e-01,-4.397995e-02,-4.397995e-02,-4.397995e-02,-6.225728e-02,-1.170411e-01,-1.472965e-01,-7.632328e-02,-6.225728e-02,-1.785287e-01,-1.842068e-01,-9.872481e-02,-2.931789e-01,-1.472965e-01,-6.225728e-02,-7.632328e-02,-4.397995e-02,-1.951311e-01,-2.390457e-01,-7.632328e-02,-2.344991e-01,-5.222330e-01,-6.225728e-02,-2.004012e-01,-2.204155e-01,-3.008759e-01,-4.273738e-01
25%,-9.910017e-01,-6.102161e-01,-4.956156e-01,-0.615974,-9.095586e-01,-7.846144e-01,-8.344418e-01,-7.480890e-01,-6.951603e-01,-2.251887e-01,-4.325358e-01,-9.071781e-01,-1.127799e+00,-2.773501e-01,-5.554640e-01,-7.938315e-01,-5.409736e-01,-6.284838e-01,-0.234499,-1.403034e-01,-4.397995e-02,-4.397995e-02,-4.397995e-02,-6.225728e-02,-1.170411e-01,-1.472965e-01,-7.632328e-02,-6.225728e-02,-1.785287e-01,-1.842068e-01,-9.872481e-02,-2.931789e-01,-1.472965e-01,-6.225728e-02,-7.632328e-02,-4.397995e-02,-1.951311e-01,-2.390457e-01,-7.632328e-02,-2.344991e-01,-5.222330e-01,-6.225728e-02,-2.004012e-01,-2.204155e-01,-3.008759e-01,-4.273738e-01
50%,1.385629e-02,9.534626e-02,3.958041e-01,0.066665,-2.402144e-01,-1.120878e-01,1.151902e-01,-1.543169e-01,1.032722e-01,-2.251887e-01,-4.325358e-01,1.754697e-03,8.866831e-01,-2.773501e-01,-5.554640e-01,1.996086e-01,-2.095433e-01,-3.878669e-01,-0.234499,-1.403034e-01,-4.397995e-02,-4.397995e-02,-4.397995e-02,-6.225728e-02,-1.170411e-01,-1.472965e-01,-7.632328e-02,-6.225728e-02,-1.785287e-

### Training

In [28]:
models = {
    "                     Linear Regression": LinearRegression(),
    "                   K-Nearest Neighbors": KNeighborsRegressor(),
    "                        Neural Network": MLPRegressor(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "   Support Vector Machine (RBF Kernel)": SVR(),
    "                         Decision Tree": DecisionTreeRegressor(),
    "                         Random Forest": RandomForestRegressor(),
    "                               XGBoost": XGBRegressor(verbose=0),
    "                              LightGBM": LGBMRegressor(verbose=0),
    "                              CatBoost": CatBoostRegressor(verbose=0)
}

In [29]:
for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                     Linear Regression trained.
                   K-Nearest Neighbors trained.


/home/aditya/miniconda3/envs/dsenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


                        Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                         Random Forest trained.
                               XGBoost trained.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

/home/aditya/miniconda3/envs/dsenv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:02:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


                              CatBoost trained.


### Results

In [30]:
for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_test, y_test)))

                     Linear Regression R^2 Score: -0.06537
                   K-Nearest Neighbors R^2 Score: -0.12779
                        Neural Network R^2 Score: -0.13027
Support Vector Machine (Linear Kernel) R^2 Score: 0.11583
   Support Vector Machine (RBF Kernel) R^2 Score: 0.11982
                         Decision Tree R^2 Score: -1.88689
                         Random Forest R^2 Score: -0.28636
                               XGBoost R^2 Score: -0.55300
                              LightGBM R^2 Score: -0.05204
                              CatBoost R^2 Score: -0.12543


In [34]:
svm_model = SVR(C=15.0)
svm_model.fit(X_train, y_train)
print("Support Vector Machine R^2: {:.5f}".format(svm_model.score(X_test, y_test)))

Support Vector Machine R^2: 0.12467
